In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob, os
import tensorflow_addons as tfa
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from PIL import Image
from tensorflow.keras.applications import EfficientNetB7, inception_v3

In [2]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('GPU device NOT found')
else:
  print('Found GPU at: {}'.format(device_name))

In [3]:
# def auto_select_accelerator():
#     try:
#         tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
#         tf.config.experimental_connect_to_cluster(tpu)
#         tf.tpu.experimental.initialize_tpu_system(tpu)
#         strategy = tf.distribute.experimental.TPUStrategy(tpu)
#         print("Running on TPU:", tpu.master())
#     except ValueError:
#         strategy = tf.distribute.get_strategy()
#     print(f"Running on {strategy.num_replicas_in_sync} replicas")
    
#     return strategy

In [4]:
benign = '../input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/benign/'
malign = '../input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/malignant/'
normal = '../input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/normal/'

In [5]:
def load_image(image_path,label):
    img = tf.io.read_file(image_path)
    img = tf.io.decode_jpeg(img, channels=3)
    img = Resizing(299, 299)(img)
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    return img,label

def remove_mask(list_):
    final_list = []
    for i in list_:
        if('mask' in i):
            continue
        else:
            final_list.append(i)
    return final_list

In [6]:
benign_image_path = os.listdir(benign)
benign_image_path = [benign+x for x in benign_image_path ]
benign_image_path = remove_mask(benign_image_path)



malign_image_path =  os.listdir(malign)
malign_image_path = [malign+x for x in malign_image_path ]
malign_image_path = remove_mask(malign_image_path)



normal_image_path = os.listdir(normal)
normal_image_path = [normal+x for x in normal_image_path ]
normal_image_path = remove_mask(normal_image_path)


In [7]:
img = np.array(Image.open(benign_image_path[0]))
plt.imshow(img)

In [8]:
benign_labels = len(benign_image_path) * [0]
malign_labels = len(malign_image_path) * [1]
normal_labels = len(normal_image_path) * [2]

In [9]:
#combining the dataset
total_paths = benign_image_path  + malign_image_path + normal_image_path
total_labels = benign_labels + malign_labels + normal_labels

In [11]:
BUFFER_SIZE = 1000
BATCH_SIZE = 64
dataset = tf.data.Dataset.from_tensor_slices((total_paths,total_labels)).map(load_image)

dataset = dataset.shuffle(BUFFER_SIZE).batch(1)

In [12]:
# #dividing in train and test
valid_dataset = dataset.take(200) 
train_dataset = dataset.skip(200)

In [13]:
def CNN_Model():
    base_model = tf.keras.applications.InceptionV3(include_top=False,weights='imagenet', input_shape= (299, 299,3))
    base_model.trainable = False
    
    x = base_model.output

    # 1st conv block
    x = Conv2D(256, 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = GlobalAveragePooling2D(keepdims = True)(x)

    # 2nd conv block
    x = Conv2D(128, 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = GlobalAveragePooling2D(keepdims = True)(x)

    x = Dense(32, activation = 'relu')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(.2)(x)

    x = Dense(3, 'softmax')(x)
    
    final_model = keras.models.Model(inputs = base_model.input, outputs = x)
    
    return final_model

In [14]:
earlyStop = keras.callbacks.EarlyStopping(patience=60) 
best_model = keras.callbacks.ModelCheckpoint(filepath='best_model.h5', save_best_only=True) 
tqdm_callback = tfa.callbacks.TQDMProgressBar()

with tf.device('/gpu:0'):
    model = CNN_Model()
    model.compile(optimizer= keras.optimizers.Adam(), loss = keras.losses.sparse_categorical_crossentropy, 
                  metrics = [keras.metrics.SparseCategoricalAccuracy()])

    history = model.fit(train_dataset , epochs = 10, validation_data = valid_dataset, batch_size=64, callbacks = [earlyStop,best_model, tqdm_callback ])

In [15]:
plt.plot(history.history['sparse_categorical_accuracy'])
plt.plot(history.history['val_sparse_categorical_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()